# DATA-641
Max Calzada

**Exercise 1** (35 points) 

Given a collection of COVID-19-related posts from various social media
platforms such as Twitter, Facebook, Instagram, etc., our task will be to pre-process, and
generate various types of features that can be used to train a model and classify a post as either
real or fake. For HWK2 answer the following questions.

(a) (1 points) 

Access and import the data from the TrainLabels.csv file.

In [3]:
!pip install textstat
!pip install gensim
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Code adapted from 
  # https://github.com/swarooprm/DQI/blob/master/1-2-3.ipynb

import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize # separates words into separate strings
from nltk.tokenize import sent_tokenize # separates sentences into separate strings
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer # identifies root word ("running" and "ran" are forms of "run")
nltk.download('averaged_perceptron_tagger') # tags parts of speech (preposition, determiner, ...)

# import BeautifulSoup
# from BeautifulSoup import BeautifulStoneSoup
import cgi
import html
import spacy
import textstat

from collections import Counter # counts words, stores them in dictionary
from itertools import chain # Groups all the iterables together and produces a single iterable as output

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from spacy.lang.en.examples import sentences

# Code based off of:
# https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab # Max: Should I delete this line?
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

import bs4 as bs
import urllib.request 
import matplotlib
import re
import nltk
from gensim.models import Word2Vec, KeyedVectors #To load the model
import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
tweets = pd.read_csv('TrainLabels.csv')

(b) For all posts within a category (real or fake), determine the total number of unique words, average words per post, and average characters per post. What do you observe about real news posts and fake news posts in terms of both characters per post and words per post?

In [10]:
# Collaborated with Dalia Habiby for (b)

def tweet_stats(df):
    
    a = df.label.unique()
    b= [a[0], a[1]]

    for label in b:
        newdf= df[df['label'] == label]
        post_col = newdf['tweet']
        post_lst = post_col.tolist()
        string = ' '.join(post_lst)
        words = string.split()
        unique_words = len(set(words))
        print(label, "category unique words", unique_words)
    
        w_per_post = []
        chr_per_post = []

        for post in post_lst:
            
            w_per_post.append(len(post.split()))
            chr_per_post.append(len(post))
    
        print(label, "average words", np.mean(w_per_post))
        print(label, "average characters", np.mean(chr_per_post))

tweet_stats(tweets)

real category unique words 17502
real average words 31.676488095238096
real average characters 215.0613095238095
fake category unique words 16396
fake average words 21.865359477124183
fake average characters 144.8875816993464


(c) (15 points) For all posts use NLTK or any other NLP Python module and perform the
following:

In [11]:
import string

# This is a standard pre-processing function
def clean_text(str_list, lemmatize = True):
    clean_list = [] # empty set
    
    for text in str_list:
        # to drop pound sign from hash tags
        text = re.sub(r'#', '', text) # gets rid of #

        # Collaborated with Dalia Habiby for the following 5 lines
        text = re.sub(r"http\S+", "", text) # c (ii) # Remove urls form text
        text = re.sub(r"@ \S+", "", text) # c (ii) # remove words starting with @
        text = re.sub(r"@\S+", "", text) # c (ii) # remove words starting with @
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = html.unescape(text) # ? # c (v) # Convert XML entities into characters

        text = text.lower() # (c) iv # lower case
        words = word_tokenize(text) # tokenizes word
        clean_words = []
        
        for word in words:            
            # drop words with fewer than 2 characters; drop any punctuation "words"
            if (len(word) > 1) and (re.match(r'^\w+$', word)):

                if lemmatize==True: # (c) iii # Lemmatization
                    lemmatizer=WordNetLemmatizer()
                    word1 = lemmatizer.lemmatize(word)
                    #if word!=word1:
                        #print(word1)
                        #print(word)
                    
                clean_words.append(word1)
        clean_text = ' '.join(clean_words)
        clean_list.append(clean_text)
    
    return clean_list

In [12]:
tweets['clean_tweet'] = clean_text(tweets['tweet'])

In [15]:
# (c) i
  # Remove all English stopwords

tweets['sw-r_ct'] = tweets['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

(d) Extract uni-grams and bi-grams derived from the bag of words representation of each pre-processed social media post.

In addition, encode these features as TF-IDF values.

In [16]:
# "bi-gram text representation"
bi_gram_text_rep = CountVectorizer(lowercase = True,  
                        stop_words = 'english',
                        ngram_range = (1,2))

bow_rep = bi_gram_text_rep.fit_transform(tweets['sw-r_ct'])

# TF-IDF
tfidf = TfidfVectorizer(lowercase = True,
                        stop_words = 'english',
                        ngram_range = (1,2))

tf_idf_rep = tfidf.fit_transform(tweets['sw-r_ct'])

bow_rep.shape, tf_idf_rep.shape

((6420, 73068), (6420, 73068))

(e) Extract features that reflect text understandibility using the different readability metrics. These features are based on properties such as words per sentence, characters per word, syllables per word, number of complex words, etc.

In [19]:
# flesch
flesch = [textstat.flesch_reading_ease(tweet) for tweet in tweets['tweet']]

# Automated Readability Index
ari = [textstat.automated_readability_index(tweet) for tweet in tweets['tweet']]

# Readability Consensus based upon other readability tests
consensus = [textstat.text_standard(tweet) for tweet in tweets['tweet']]

# McAlpine EFLAW Readability Score
eflaw = [textstat.mcalpine_eflaw(tweet) for tweet in tweets['tweet']]

# Reading Time
readtime = [textstat.reading_time(tweet) for tweet in tweets['tweet']]

# Syllable Count
syllable = [textstat.syllable_count(tweet) for tweet in tweets['tweet']]

# Lexicon Count
lexicon = [textstat.lexicon_count(tweet) for tweet in tweets['tweet']]

(f) Calculate the average readibility scores of the real post as well as the fake posts for each readibility grade. What do you observe?

In [1]:
!pip install https://github.com/andreasvc/readability/tarball/master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 35.0 kB 3.3 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.1-py3-none-any.whl size=36403 sha256=33dd2e2525b5089693018911e224a6ed1e1980c703f362b9a507578a88f38660
  Stored in directory: /tmp/pip-ephem-wheel-cache-rvu8h75c/wheels/2b/9e/87/3bbf4e61d79902dcb8a4315e715983319cfbd1bdb08f056d89
Successfully built readability


In [28]:
# Code based off of:
  # https://pypi.org/project/readability/

import readability

def tweet_readability(df):
    
    a = df.label.unique()
    b= [a[0], a[1]]

    for label in b:
        newdf= df[df['label'] == label]
        post_col = newdf['tweet']
        post_lst = post_col.tolist()

        for post in post_lst:
            
            results = readability.getmeasures(post, lang='en')
    
        print(label, "Flesch Reading Ease:", results['readability grades']['FleschReadingEase'])

tweet_readability(tweets)

real Flesch Reading Ease: 52.66918918918922
fake Flesch Reading Ease: 112.08500000000001


Fake posts tend to be more easily readible than real posts.